This file is intended to be ran only once for the initial setup and loading of raw data
- Datasource: https://wildfire.alberta.ca/resources/historical-data/historical-wildfire-database.aspx

In [2]:
# First, scrape the web
# Import Splinter, BeautifulSoup, and Pandas
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from config import dbpassword
from config import awspassword




====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\Steven\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [6]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\Steven\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


In [7]:
# Visit the site
url = 'https://wildfire.alberta.ca/resources/historical-data/historical-wildfire-database.aspx'
browser.visit(url)

# Optional delay for loading the page

browser.is_element_present_by_css('div.list_text', wait_time=1)



False

In [9]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
data = soup(html, 'html.parser')

slide_elem = data.select_one('div.table-responsive')


In [10]:
# Obtain all (relative) links that end with .csv extension and put it into a dataframe
links_df =pd.DataFrame(columns=['RelLink'])
for link in slide_elem.find_all('a'):
    if link.get('href')[-3:] == 'csv':
        links_df = links_df.append({'RelLink': link.get('href')},ignore_index=True)
links_df

,RelLink
0,documents/fires_2006to2018.csv
1,documents/fires_1996to2005.csv
2,documents/fires_1983to1995.csv
3,documents/fires_1961to1982.csv


In [11]:
# Create base url from each relative url and name-to-be for each database table
links_df['BaseLink']=[f'https://wildfire.alberta.ca/resources/historical-data/{link}' for link in links_df['RelLink']]
links_df['FileName'] = [link[10:len(link)-4:1] for link in links_df['RelLink']]

In [12]:
# Final Check
links_df

,RelLink,BaseLink,FileName
0,documents/fires_2006to2018.csv,https://wildfire.alberta.ca/resources/historic...,fires_2006to2018
1,documents/fires_1996to2005.csv,https://wildfire.alberta.ca/resources/historic...,fires_1996to2005
2,documents/fires_1983to1995.csv,https://wildfire.alberta.ca/resources/historic...,fires_1983to1995
3,documents/fires_1961to1982.csv,https://wildfire.alberta.ca/resources/historic...,fires_1961to1982


Initilize Database Connection Strings

In [13]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '$SPARK_VERSION-bin-hadoop2.7.tgz'


^C


Exception: Unable to find py4j, your SPARK_HOME may not be configured correctly

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LMPT-Forest-Fires").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
connection_string = 'lmpt-finalproject.coke2w4vs8wf.us-east-2.rds.amazonaws.com'
password = dbpassword #obtain this later from an add
database_name = 'postgres'

# Configure settings for RDS
mode = "append"
jdbc_url=f"jdbc:postgresql://{connection_string}:5432/{database_name}"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

Combine both Database and Webscrape Elements

In [ ]:
# Import CSVs into Datatbase via a For Loop
from pyspark import SparkFiles
for i,j in zip(links_df['BaseLink'],links_df['FileName']):
    url=i
    df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)
    df.write.jdbc(url=jdbc_url, table=j, mode=mode, properties=config)